# Basic RNN
- Objective: to understand basics of RNN & LSTM

## Recurrent Neural Networks
- Feedforward neural networks (e.g. MLPs and CNNs) are powerful, but they are not optimized to handle "sequential" data
- In other words, they do not possess "memory" of previous inputs
- For instance, consider the case of translating a corpus. You need to consider the **"context"** to guess the next word to come forward

<img src="http://2.bp.blogspot.com/-9GIdV292xV4/UwOIy6B6koI/AAAAAAAAHi4/X6UGlyHI-_U/s1600/tumblr_ms5qzpFY671r9nm7io1_500.gif" style="width: 500px"/>

<br>
- RNNs are suitable for dealing with sequential format data since they have **"recurrent"** structure
- To put it differently, they keep the **"memory"** of earlier inputs in the sequence
</br>
<img src="http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg" style="width: 600px"/>

<br>
- However, in order to reduce the number of parameters, every layer of different time steps shares same parameters
</br>

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="width: 600px"/>

## Load Dataset

In [1]:
import numpy as np
import tensorflow as tf

from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 1s 0us/step


In [5]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [6]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

In [8]:
y_train = y_data[:1395]
y_test = y_data[1395:]

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


## 1. Vanilla RNN
- Vanilla RNNs have a simple structure
- However, they suffer from the problem of "long-term dependencies"
- Hence, they are not able to keep the **sequential memory" for long

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" style="width: 600px"/>

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [13]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46)) # 46개의 OUTPUT 을 가지는 DENSE 측
    model.add(Activation('softmax')) # softmax 의 activation
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [16]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 100, batch_size = 50, verbose = 1)

In [17]:
model.fit(X_train, y_train)

Epoch 1/100
28/28 [==============================] - 0s 11ms/step - loss: 3.5011 - accuracy: 0.3161
Epoch 2/100
28/28 [==============================] - 0s 12ms/step - loss: 1.8918 - accuracy: 0.6738
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.3323 - accuracy: 0.6903
Epoch 4/100
28/28 [==============================] - 0s 12ms/step - loss: 1.2089 - accuracy: 0.7147
Epoch 5/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1794 - accuracy: 0.7147
Epoch 6/100
28/28 [==============================] - 0s 12ms/step - loss: 1.1667 - accuracy: 0.7147
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1596 - accuracy: 0.7147
Epoch 8/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1562 - accuracy: 0.7147
Epoch 9/100
28/28 [==============================] - 0s 13ms/step - loss: 1.1540 - accuracy: 0.7147
Epoch 10/100
28/28 [==============================] - 0s 12ms/step - loss: 1.1517 - accuracy: 0.7147

28/28 [==============================] - 0s 10ms/step - loss: 0.9496 - accuracy: 0.7333
Epoch 82/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9511 - accuracy: 0.7305
Epoch 83/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9349 - accuracy: 0.7369
Epoch 84/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9347 - accuracy: 0.7376
Epoch 85/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9317 - accuracy: 0.7355
Epoch 86/100
28/28 [==============================] - 0s 13ms/step - loss: 0.9591 - accuracy: 0.7312
Epoch 87/100
28/28 [==============================] - 0s 11ms/step - loss: 0.9264 - accuracy: 0.7362
Epoch 88/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9300 - accuracy: 0.7283
Epoch 89/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9237 - accuracy: 0.7290
Epoch 90/100
28/28 [==============================] - 0s 12ms/step - loss: 0.9295 - accuracy: 0.7305
Epo

In [18]:
y_pred = model.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
12/12 [==============================] - 0s 4ms/step


In [20]:
y_test_ = np.argmax(y_test, axis = 1)

In [21]:
print(accuracy_score(y_pred, y_test_))

0.7445742904841403


## 2. Stacked Vanilla RNN
- RNN layers can be stacked to form a deeper network

<img src="https://lh6.googleusercontent.com/rC1DSgjlmobtRxMPFi14hkMdDqSkEkuOX7EW_QrLFSymjasIM95Za2Wf-VwSC1Tq1sjJlOPLJ92q7PTKJh2hjBoXQawM6MQC27east67GFDklTalljlt0cFLZnPMdhp8erzO" style="width: 500px"/>

In [74]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

In [75]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [77]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [78]:
print(accuracy_score(y_pred, y_test_))

0.746243739566


## 3. LSTM
- LSTM (long short-term memory) is an improved structure to solve the problem of long-term dependencies

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="width: 600px"/>

In [79]:
from keras.layers import LSTM

In [80]:
def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [81]:
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [83]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [84]:
# accuracy improves by adopting LSTM structure
print(accuracy_score(y_pred, y_test_))

0.844741235392


## 4. Stacked LSTM
- LSTM layers can be stacked as well

In [85]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [86]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [88]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [89]:
print(accuracy_score(y_pred, y_test_))

0.858096828047
